# Data Cleaning
We may have to come back to this.

In [1]:
!nvidia-smi -L

zsh:1: command not found: nvidia-smi


In [6]:
from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [7]:
import pandas as pd
import numpy as np

In [9]:
if 'google.colab' in str(get_ipython()):
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = ""
df = pd.read_csv(f"{proj_dir}data_raw/ist.csv", low_memory=False)

Using the [stroke dataset](https://trialsjournal.biomedcentral.com/articles/10.1186/1745-6215-12-101#MOESM1) we want to use these data to generate additioal samples. For a data dictionary see [the paper](https://www.researchgate.net/publication/51068185_The_International_Stroke_Trial_database).

In [88]:
df.columns = [x.lower() for x in df.columns]

In [89]:
df = df.applymap(lambda x: x.lower() if type(x) == str else x)
df = df.apply(lambda x: x.replace(to_replace=['n', 'y'], value=[0, 1]))
df = df.apply(lambda x: x.replace(to_replace=['c', 'u', 'unknown'], value=np.nan))

I am KNN inputing missing for contnous valued columns, then replacing with 0 the dummy variables.

In [90]:
df['sex'] = df.sex.replace(to_replace=['f', 'm'], value=[0, 1])
dummy_cols = ['rconsc', 'stype', 'rxhep', 'dmajnchx', 'dplace', 'fplace', 'nccode', 'country']
dummy_df = pd.get_dummies(df[dummy_cols])
dummy_df = dummy_df.fillna(0, inplace=True)

In [91]:
df = df.apply(pd.to_numeric, errors='coerce')
numeric_df = df.drop(columns=dummy_cols)
misc_drop = ["rdate", "dsidex", "dnostrkx", "drshd", "ddeadx", "fdeadx"] # these are not helpful and causing problems

In [92]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)

In [93]:
numeric_df = numeric_df.drop(columns=misc_drop)
numeric_df = pd.DataFrame(imputer.fit_transform(numeric_df),columns = numeric_df.columns)
df = pd.concat([numeric_df, dummy_df])

In [94]:
df.to_csv(f"{proj_dir}data_clean/clean.csv", index=False)